The code below serves as a reference for exporting features from Vertex Feature Store Legacy (1.0) to the new and fast Vertex Feature Store 2.0.
This script is not an official Google release and may contain bugs.
mateuswagner@google.com

Ref.: 
 - https://cloud.google.com/vertex-ai/docs/featurestore#migrate
 - https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official

PART-1: 

    - Install required packages
    - Initialize clients
    - Export EntityType to Bigquery
    - Show exported rows count and schema
    
PART-2

    - Create VFS 2.0 feature online store
    - Create Feature store
    - Start on-demand sync!
    - Check FeatureView Syncs status

    


#### Install required packages

In [ ]:
%pip install -q --upgrade google-cloud-aiplatform google-cloud-bigquery

In [ ]:
import logging
from google.cloud import bigquery
from google.cloud.aiplatform_v1beta1 import (FeaturestoreServiceClient,
                                             FeatureValueDestination,
                                             FeatureSelector,
                                             ExportFeatureValuesRequest,
                                             ListFeatureViewSyncsRequest,
                                             types)
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient, FeatureOnlineStoreServiceClient)

from google.cloud.aiplatform_v1beta1.types import NearestNeighborQuery

from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store as feature_online_store_pb2

from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store_admin_service as \
    feature_online_store_admin_service_pb2

from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store_service as feature_online_store_service_pb2

from google.cloud.aiplatform_v1beta1.types import \
    feature_view as feature_view_pb2
import time


In [11]:
# Settings

PROJECT_ID = "matt-demos"  # Change it
REGION_ID = "us-central1" # Change it
FEATURE_STORE_ID = "legacy_fs_1" # Change it
ENTITYTYPE_ID = "benchmark_3emb_1000xfloat64" # Change it
BIGQUERY_DEST_DATASET_ID = "featurestore_ds" # Change it
BIGTABLE_MAX_NODE_COUNT = 2 # Change it
BIGTABLE_CPU_UTILIZATION_TARGET = 50 # Default
FEATURE_ONLINE_STORE_ID = "featurestore_4" # Change it
FEATURE_ONLINE_STORE_EMBEDDING_MNGMT = True # Change it
CRON_SCHEDULE = "TZ=America/Los_Angeles 56 * * * *"  # Change it!
logging.getLogger().setLevel(logging.INFO) # set to DEBUG during trobleshooting
DEFAULT_TIMEOUT = 3600


In [ ]:
# Aux variables

parent_id = f'projects/{PROJECT_ID}/locations/{REGION_ID}'

entitytype_id_path = f'{parent_id}/featurestores/{FEATURE_STORE_ID}/entityTypes/{ENTITYTYPE_ID}'

entity_type_short_id = entitytype_id_path.split("/")[-1]

bigquery_output_url = f'bq://{PROJECT_ID}.{BIGQUERY_DEST_DATASET_ID}.{entity_type_short_id}'

entity_id_column = f'entity_type_{entity_type_short_id}'

feature_online_stores_name = f"{parent_id}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"

feature_view_id = entity_type_short_id

feature_view_full = f"{feature_online_stores_name}/featureViews/{feature_view_id}"


### Initialize clients

In [ ]:

# Get vertex FS client (using local credentials)
fs_client = FeaturestoreServiceClient(
    client_options = {
        "api_endpoint": f'{REGION_ID}-aiplatform.googleapis.com'
    }
)

# Get bigquery client
bq_client = bigquery.Client()

# Get vertex Feature store 2.0 Online store admin client
admin_vfs2_client = FeatureOnlineStoreAdminServiceClient(
    client_options = {
        "api_endpoint": f'{REGION_ID}-aiplatform.googleapis.com'
    }
)

### Export EntityType (VFS Legacy) to Bigquery

In [ ]:
destination = FeatureValueDestination()
destination.bigquery_destination.output_uri = bigquery_output_url

feature_selector = FeatureSelector()
feature_selector.id_matcher.ids = ['*']

# request job
logging.info("Exporting features")

operation = fs_client.export_feature_values(
    ExportFeatureValuesRequest(
        entity_type = entitytype_id_path,
        destination = destination,
        feature_selector = feature_selector,
        full_export = types.ExportFeatureValuesRequest.FullExport()
    ),
    timeout = DEFAULT_TIMEOUT
)

#  Wait export operation to finish
operation.result()

logging.info(f'Entity type "{entity_type_short_id}" from feature store \
"{feature_store_id}" located at "{REGION_ID}" of project "{PROJECT_ID}" to Bigquery table "{bigquery_output_url}"')

### Show exported rows count and schema

In [ ]:
table_id = bq_client.get_table(f'{BIGQUERY_DEST_DATASET_ID}.{ENTITYTYPE_ID}')

print(f'The table {table_id} has {table_id.num_rows} \
rows and {len(table_id.schema)} columns.\n\n Schema: {table_id.schema}')

## Part 2

### Create a new feature online store (VFS 2.0) 

In [ ]:
# Request a new online store

# Define feature online store settings
online_store_config = feature_online_store_pb2.FeatureOnlineStore(
    optimized = feature_online_store_pb2.FeatureOnlineStore.Optimized(),
    bigtable = feature_online_store_pb2.FeatureOnlineStore.Bigtable(
        auto_scaling = feature_online_store_pb2.FeatureOnlineStore.Bigtable.AutoScaling(
            min_node_count = 1, 
            max_node_count = BIGTABLE_MAX_NODE_COUNT, 
            cpu_utilization_target = BIGTABLE_CPU_UTILIZATION_TARGET
        )
    ),
    embedding_management=feature_online_store_pb2.FeatureOnlineStore.EmbeddingManagement(
        enabled = FEATURE_ONLINE_STORE_EMBEDDING_MNGMT
    )
)

create_store_lro = admin_vfs2_client.create_feature_online_store(
    feature_online_store_admin_service_pb2.CreateFeatureOnlineStoreRequest(
        parent = parent_id,
        feature_online_store_id = FEATURE_ONLINE_STORE_ID,
        feature_online_store = online_store_config,
    )
)
# Wait
print(create_store_lro.result())

### Create a FeatureView

In [ ]:
big_query_source = feature_view_pb2.FeatureView.BigQuerySource(
    uri = bigquery_output_url, 
    entity_id_columns = [entity_id_column]
)

sync_config = feature_view_pb2.FeatureView.SyncConfig(cron=CRON_SCHEDULE)

create_view_lro = admin_vfs2_client.create_feature_view(
    feature_online_store_admin_service_pb2.CreateFeatureViewRequest(
        parent = feature_online_stores_name,
        feature_view_id = feature_view_id,
        feature_view = feature_view_pb2.FeatureView(
            big_query_source = big_query_source,
            sync_config=sync_config,
        )
    )
)

print(create_view_lro.result())


### Start on-demand sync!

In [ ]:
sync_response = admin_vfs2_client.sync_feature_view(
    feature_view=feature_view_full
)

while True:
    feature_view_sync = admin_vfs2_client.get_feature_view_sync(
        name=sync_response.feature_view_sync
    )
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

### Check FeatureView Syncs status

In [ ]:
# Initialize request argument(s)
request = ListFeatureViewSyncsRequest(
    parent=feature_view_full,
)

# Make the request
page_result = admin_vfs2_client.list_feature_view_syncs(request=request)

# Handle the response
for response in page_result:
    print(response)

### Uncomment the following cell to delete your Feature Online Store

In [ ]:
#op = admin_vfs2_client.delete_feature_online_store(
#    name=f"projects/{PROJECT_ID}/locations/{REGION_ID}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}",
#    force=True,
#)
